In [27]:
# 텍스트 LLM 모델 : 단입 입력 -> 단일 출력
# chat_model : 대화형
# 차이는... chat_model은 gpt-3.5-turbo를, llm은 text-davinci-003을 쓰는데, gpt-3.5-turbo가 가격이 1/10 임
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

InvalidRequestError: The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

In [ ]:
# 메시지타입을 활용한 Chat 모델 : 기존의 대화 히스토리를 기반으로 질문에 대한 답변을 출력하는 시나리오
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian",
    ),
    AIMessage(content="Ciao, mi chiamo Paolo!",),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?",),
]

chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono un esperto di geografia. Posso aiutarti con altre domande?')

In [ ]:
# PromptTemplate : String 형태로 프롬프트를 세팅
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}.")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [ ]:
# ChatPromptTemplate : Message 형태로 프롬프트를 세팅
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language="Korean",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 대략 16,000km입니다. 제 이름은 소크라테스입니다. 어떻게 도와드릴까요?')

In [ ]:
# OutputParser : LLM 의 응답을 변형하고자 할때 사용(DB화, list, dict, tuple 등)
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    # parse method를 반드시 구현해야함
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("how, are, you")

['how', 'are', 'you']

In [ ]:
# ChatPromptTemplate의 결과를 OutputParser를 통해 변형(예제에서는 list에 저장)
template = ChatPromptTemplate.from_messages([
    ('system', 'You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in uppercase. Do NOT reply with anythin else.'),
    ('human', '{question}')
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['MERCURY', 'VENUS', 'EARTH', 'MARS', 'JUPITER', 'SATURN', 'URANUS', 'NEPTUNE']

In [ ]:
# 위의 방법(template 생성, format, prompt, predict, parse)이 복잡하므로,
# template, Language Model, OutputParser만 있으면 되는 방법을 소개 => chain

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the three kingdoms?"
})

['ANIMALIA', 'PLANTAE', 'FUNGI']

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
# streaming, callbacks는 응답작성 과정을 보여주기 위한 파라미터
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ('human', "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat 

In [29]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ('human', "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

# RunnableMap 정의. chef_chain의 출력을 "recipe"라는 이름으로 받아서, veg_chain에 입력으로 제공
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({
    "cuisine": "indian",
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped for garnish
- Cooked rice or naan bread for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerat

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can replace the chicken with a suitable alternative. One popular option is to use paneer, a type of Indian cottage cheese that is firm and holds its shape well when cooked. Here's how you can prepare the paneer as a substitute for chicken in this recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized cubes\n- 1 cup plain yogurt\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon ginger, minced\n- 1 can (14 oz) tomato sauce\n- 1 tablespoon garam masala\n- 1 teaspoon ground cumin\n- 1 teaspoon ground coriander\n- 1 teaspoon paprika\n- 1/2 teaspoon turmeric\n- 1/2 teaspoon cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- Fresh cilantro, chopped for garnish\n- Cooked rice or naan bread for serving\n\nInstructions:\n1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced gar

In [32]:
# FewShotPromptTemplate : 더 나은 대답을 할 수 있도록 모델에게 어떻게 대답해야 하는지에 대한 예제를 형식화
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

examples = [
    { 
        "question": "What do you know about France?", 
        "answer": """ 
        Here is what I know: 
        Capital: Paris 
        Language: French 
        Food: Wine and Cheese 
        Currency: Euro 
        """, 
    }, 
    { 
        "question": "What do you know about Italy?", 
        "answer": """ 
        I know this: 
        Capital: Rome 
        Language: Italian 
        Food: Pizza and Pasta 
        Currency: Euro 
        """, 
    }, 
    { 
        "question": "What do you know about Greece?", 
        "answer": """ 
        I know this: 
        Capital: Athens 
        Language: Greek 
        Food: Souvlaki and Feta Cheese 
        Currency: Euro 
        """, 
    }, 
]

# chat.predict("What do you know about France?")

# DB나 파일 등에서 가져올 수도 있으므로, 예제 형식지정 도구를 만듬
example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)
# 이것과 동일함 example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")


# 예제 프롬프트, 예제를 지정하고, 질문을 suffix로 세팅, 입력값을 input_variables로 지정
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Republic of Korea")

chain = prompt | chat

chain.invoke({
    "country": "Republic of Korea"
})

AI: 
        Here is what I know: 
        Capital: Seoul 
        Language: Korean 
        Food: Kimchi and Bibimbap 
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        Here is what I know: \n        Capital: Seoul \n        Language: Korean \n        Food: Kimchi and Bibimbap \n        Currency: South Korean Won')

In [33]:
# FewShotChatMessagePromptTemplate 을 사용하여 조금 수정해 봄
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

examples = [
    { 
        "country": "France", 
        "answer": """ 
        Here is what I know: 
        Capital: Paris 
        Language: French 
        Food: Wine and Cheese 
        Currency: Euro 
        """, 
    }, 
    { 
        "country": "Italy", 
        "answer": """ 
        I know this: 
        Capital: Rome 
        Language: Italian 
        Food: Pizza and Pasta 
        Currency: Euro 
        """, 
    }, 
    { 
        "country": "Greece",
        "answer": """ 
        I know this: 
        Capital: Athens 
        Language: Greek 
        Food: Souvlaki and Feta Cheese 
        Currency: Euro 
        """, 
    }, 
]

example_prompt = ChatPromptTemplate.from_messages([
    ('human', "What do you know about {country}?"),
    ('ai', "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a geography expert"),
    example_prompt,
    ('human', "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({
    "country": "Spain"
})

I know this: 
        Capital: Madrid 
        Language: Spanish 
        Food: Paella and Tapas 
        Currency: Euro

AIMessageChunk(content='I know this: \n        Capital: Madrid \n        Language: Spanish \n        Food: Paella and Tapas \n        Currency: Euro')

In [37]:
# LengthBasedExampleSelector : 길이 기반으로 예제를 선택할 수 있음(프롬프트 크기를 줄여서 돈 절약)
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

examples = [
    { 
        "question": "What do you know about France?", 
        "answer": """ 
        Here is what I know: 
        Capital: Paris 
        Language: French 
        Food: Wine and Cheese 
        Currency: Euro 
        """, 
    }, 
    { 
        "question": "What do you know about Italy?", 
        "answer": """ 
        I know this: 
        Capital: Rome 
        Language: Italian 
        Food: Pizza and Pasta 
        Currency: Euro 
        """, 
    }, 
    { 
        "question": "What do you know about Greece?", 
        "answer": """ 
        I know this: 
        Capital: Athens 
        Language: Greek 
        Food: Souvlaki and Feta Cheese 
        Currency: Euro 
        """, 
    }, 
]

# chat.predict("What do you know about France?")

# DB나 파일 등에서 가져올 수도 있으므로, 예제 형식지정 도구를 만듬
example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)
# 이것과 동일함 example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# 필요한 예제를 고르기 위한 길이 기반 선택기 세팅 (1.원하는 답변을 얻기 위해 2.돈 절약. 긴prompt=돈)
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100,
)

# 예제 프롬프트, 예제를 지정하고, 질문을 suffix로 세팅, 입력값을 input_variables로 지정
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples=examples
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Republic of Korea")

'\n    Human: What do you know about France?\n    AI:  \n        Here is what I know: \n        Capital: Paris \n        Language: French \n        Food: Wine and Cheese \n        Currency: Euro \n        \n\n\nHuman: What do you know about Republic of Korea?'

In [45]:
# 랜덤 예제 선택기 구현
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

examples = [
    { 
        "question": "What do you know about France?", 
        "answer": """ 
        Here is what I know: 
        Capital: Paris 
        Language: French 
        Food: Wine and Cheese 
        Currency: Euro 
        """, 
    }, 
    { 
        "question": "What do you know about Italy?", 
        "answer": """ 
        I know this: 
        Capital: Rome 
        Language: Italian 
        Food: Pizza and Pasta 
        Currency: Euro 
        """, 
    }, 
    { 
        "question": "What do you know about Greece?", 
        "answer": """ 
        I know this: 
        Capital: Athens 
        Language: Greek 
        Food: Souvlaki and Feta Cheese 
        Currency: Euro 
        """, 
    }, 
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

# DB나 파일 등에서 가져올 수도 있으므로, 예제 형식지정 도구를 만듬
example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)
# 이것과 동일함 example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

# 예제 프롬프트, 예제를 지정하고, 질문을 suffix로 세팅, 입력값을 input_variables로 지정
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples=examples
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Republic of Korea")

'\n    Human: What do you know about Italy?\n    AI:  \n        I know this: \n        Capital: Rome \n        Language: Italian \n        Food: Pizza and Pasta \n        Currency: Euro \n        \n\n\nHuman: What do you know about Republic of Korea?'

In [50]:
# 프롬프트를 파일에서 읽어오기
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country='Germany')

'What is the capital of Germany'

In [46]:
# 여러 프롬프트를 합치는 방법
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrr!")

In [51]:
# LLM 답변을 cache 혹은 DB에 저장하여, 같은 질문에 대한 답변은 LLM을 통하지 않고 즉시 답변 제공. 돈 절약
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# 답변을 메모리에 저장. 재부팅시 사라짐
# set_llm_cache(InMemoryCache())

# 답변이 생성되는 과정을 보여주는 기능
# set_debug(True)

# 답변을 DB에 저장
set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\

In [52]:
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\

In [ ]:
# OpenAI 사용량 체크
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

In [56]:
# MEMORY
# 1. ConversationBufferMemory : 대화 전체를 전부 저장. 이해하기 쉽고 text completion에 활용 가능. 모든 대화를 저장하므로 점점 커져서 비효율적, 비용높음
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [60]:
memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

In [62]:
# 2. ConversationBufferWindowMemory : 최신 대화만 저장. 메모리 크기 유지 가능. 최근 내용에만 집중해서 예전 대화를 기억하기 힘들다는 단점

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(return_messages=True, k=4)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [63]:
# FIFO 방식으로 오래된 대화가 삭제된 것을 확인할 수 있다.
add_message(5, 5)

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [64]:
# 3. ConversationSummaryMemory : 메시지를 요약해서 저장. 초반엔 대화보다 저장하는 내용이 길어질 수 있으나, 대화가 진행될수록 과거 내용을 버리지도 않고, 요약의 도움을 받아 토큰의 양도 줄어듬.

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm ggomdong, I live in South Korea", "Wow that is so cool")

In [65]:
add_message("South Korea is very beautiful plcae", "I wish I could go there")

In [66]:
get_history()

{'history': 'The human introduces themselves as ggomdong from South Korea. The AI responds with enthusiasm, finding it cool and expressing a wish to visit the beautiful place.'}

In [79]:
# 4. ConversationSummaryBufferMemory : 메시지의 수를 카운트하고, limit 도달시 오래된 메시지를 버리는 대신 요약함. 최신에 집중하되 과거도 참고 가능

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm ggomdong, I live in South Korea", "Wow that is so cool")

In [80]:
# 아직 max_token_limit에 도달하지 않아서, 대화가 그대로 저장됨
get_history()

{'history': [HumanMessage(content="Hi I'm ggomdong, I live in South Korea"),
  AIMessage(content='Wow that is so cool')]}

In [81]:
add_message("South Korea is very beautiful plcae", "I wish I could go there")

In [82]:
# max_token_limit에 도달하여, 오래된 메시지는 요약되어 SystemMessage로 저장
get_history()

{'history': [SystemMessage(content='The human introduces themselves as ggomdong and mentions they live in South Korea.'),
  AIMessage(content='Wow that is so cool'),
  HumanMessage(content='South Korea is very beautiful plcae'),
  AIMessage(content='I wish I could go there')]}

In [83]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [84]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as ggomdong and mentions they live in South Korea. The AI responds with admiration, saying it is cool. The human adds that South Korea is a very beautiful place.'),
  AIMessage(content='I wish I could go there'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [85]:
# 5. ConversationKGMemory : 메시지의 entity 정보를 저장. 요약하여 답변

from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(llm=llm, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hi I'm ggomdong, I live in South Korea", "Wow that is so cool")

In [86]:
memory.load_memory_variables({"input": "who is ggomdong?"})

{'history': [SystemMessage(content='On ggomdong: ggomdong lives in South Korea.')]}

In [87]:
add_message("South Korea is very beautiful plcae", "I wish I could go there")

In [89]:
memory.load_memory_variables({"input": "what does ggomdong think about South Korea?"})

{'history': [SystemMessage(content='On ggomdong: ggomdong lives in South Korea.'),
  SystemMessage(content='On South Korea: South Korea is beautiful. South Korea is in place.')]}

In [90]:
# Memory 를 chain에 적용. 
# 1. 문자열 기반 LLM에 적용하는 일반적인 방법(off-the-shelf)
# 메모리에 memory_key를 세팅하여, 템플릿에 이 값을 전달함으로써 챗봇이 기존대화를 알게 해준다.
# verbose : 프롬프트의 log를 보여줌
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Nico
    You:


> Finished chain.


'Hello Nico! How can I assist you today?'

In [91]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


"That's great to know! How can I assist you with information or tasks related to Seoul?"

In [92]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist them today.
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico.'

In [93]:
# Memory 를 chain에 적용. 
# 2. 문자열이 아닌 메시지 기반의 대화에 메모리에 저장된 대화 기록을 활용하는 방법
# return_messages : 사람/시스템/ai 등 모든 메시지를 문자열이 아닌 메시지 클래스로 바꿈
# MessagePlaceholder : 메시지의 종류나 크기 등에 무관하게 적용할 수 있게 해주는 패키지
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico

> Finished chain.


'Hello Nico! How can I assist you today?'

In [94]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?'

In [95]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?
Human: What is my name?

> Finished chain.


'Your name is Nico.'

In [96]:
# Memory 를 chain에 적용. 
# 3. LangChain Expression Language 에 수동으로 메모리 적용
# 이 방법의 문제점은 invoke할때마다 chat_history를 지정해야 한다는 점
# -> RunnablePassthrough에서 먼저 load_memory함수를 실행해서 prompt에 제공하는 방식으로 해결
# 메모리를 수동으로 관리(save/load)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [97]:
invoke_chain("My name is Nico")

content='Hello Nico! How can I assist you today?'


In [98]:
invoke_chain("What is my name?")

content='Your name is Nico.'
